# Text Analysis on gurlitt data

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [3]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,535386605666979840,43092938,Will Bern's Museum of Fine Arts accept the con...,Thu Nov 20 10:58:08 +0000 2014,0
1,535391478969675776,38402632,The Gurlitt art collection no one - and everyo...,Thu Nov 20 11:17:30 +0000 2014,0
2,535415010361487360,719898644,German and Swiss handling of Gurlitt hoard and...,Thu Nov 20 12:51:00 +0000 2014,0
3,535415869862473729,1140049158,The Gurlitt art collection no one - and everyo...,Thu Nov 20 12:54:25 +0000 2014,0
4,535425236871544832,289989248,The Gurlitt art collection no one - and everyo...,Thu Nov 20 13:31:38 +0000 2014,0
...,...,...,...,...,...
159,536848134681690112,22138756,Summary of the #Gurlitt agreement between Kuns...,Mon Nov 24 11:45:44 +0000 2014,1
160,536848139836481536,16541374,Gurlitt collection accepted by Kunstmuseum Ber...,Mon Nov 24 11:45:45 +0000 2014,1
161,536848241556721665,553661747,Swiss museum accepts looted Nazi art http://t....,Mon Nov 24 11:46:09 +0000 2014,1
162,536848662135980032,59608915,Gurlitt Collection Accepted by Kunstmuseum Ber...,Mon Nov 24 11:47:49 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [ ]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
          
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    text = [stem.stem(i) for i in text]
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [ ]:
print("original text: ", df.reply_tweet.iloc[107])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[107]))

### applied on original data (**df**)

In [ ]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [ ]:
df.head()

### applied on source tweet data (**src_tw_df**) 

In [ ]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [ ]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [ ]:
df.head(1)

### applied on source tweet data only 

In [ ]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/gurlitt/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/gurlitt/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/gurlitt/senti_source.pdf',dpi=300)

### save data to csv 

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/gurlitt/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/gurlitt/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/gurlitt/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [ ]:
import text2emotion as te

In [ ]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [ ]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emotion']=emo

In [5]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-df.csv',index=False)

In [ ]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emotion']=emo1

In [4]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-src.csv',index=False)

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-src.csv', encoding = 'utf-8', header =0)

In [3]:
src_tw_df.drop('emotion',axis=1,inplace=True)
df.drop('emotion',axis=1,inplace=True)